In [2]:
import json
import numpy as np
import openai
from tqdm import tqdm
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

print("=" * 70)
print("RAG SYSTEM SETUP")
print("=" * 70)

# Load chunks
print("\nLoading chunks...")
with open('/home/zeynkash/projects/izu_scraper/chunking/chunks.json', 'r', encoding='utf-8') as f:
    chunks = json.load(f)

print(f"✓ Loaded {len(chunks)} chunks")
print(f"\nSample chunk:")
print(json.dumps(chunks[0], indent=2, ensure_ascii=False)[:300])

RAG SYSTEM SETUP

Loading chunks...
✓ Loaded 1747 chunks

Sample chunk:
{
  "chunk_id": "8704994871202097904_0",
  "document_id": 8704994871202097904,
  "chunk_index": 0,
  "total_chunks": 1,
  "content": "Title: Student Guide\nURL: https://www.izu.edu.tr/en/international/international-students/student-guide\n\nStudent Guide Welcome to Istanbul Sabahattin Zaim Universit


In [3]:
def get_embedding(text, model="text-embedding-3-small"):
    """
    Get embedding from OpenAI API
    
    Args:
        text: Text to embed
        model: OpenAI embedding model
    
    Returns:
        numpy array of embedding
    """
    text = text.replace("\n", " ")
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# Test on one chunk
print("Testing embedding generation...")
test_embedding = get_embedding(chunks[0]['content'])
print(f"✓ Embedding generated!")
print(f"  Dimension: {len(test_embedding)}")
print(f"  Sample values: {test_embedding[:5]}")

# Estimate cost
total_tokens = sum(c['tokens'] for c in chunks)
cost = (total_tokens / 1_000_000) * 0.02  # $0.02 per 1M tokens for text-embedding-3-small
print(f"\n💰 Estimated cost: ${cost:.2f} for {total_tokens:,} tokens")

Testing embedding generation...
✓ Embedding generated!
  Dimension: 1536
  Sample values: [-0.02047703042626381, 0.05622651427984238, -0.013715344481170177, 0.01983712427318096, -0.02147955261170864]

💰 Estimated cost: $0.02 for 1,021,515 tokens


In [4]:
import time

def generate_embeddings_batch(chunks, batch_size=100, model="text-embedding-3-small"):
    """
    Generate embeddings for all chunks in batches
    
    Args:
        chunks: List of chunk objects
        batch_size: Number of chunks to process at once
        model: OpenAI embedding model
    
    Returns:
        numpy array of embeddings
    """
    embeddings = []
    
    print(f"Generating embeddings in batches of {batch_size}...")
    
    for i in tqdm(range(0, len(chunks), batch_size), desc="Processing batches"):
        batch = chunks[i:i + batch_size]
        texts = [chunk['content'].replace("\n", " ") for chunk in batch]
        
        try:
            # API call for batch
            response = openai.embeddings.create(
                input=texts,
                model=model
            )
            
            # Extract embeddings
            batch_embeddings = [item.embedding for item in response.data]
            embeddings.extend(batch_embeddings)
            
            # Rate limit safety (optional)
            time.sleep(0.1)
            
        except Exception as e:
            print(f"\nError in batch {i}-{i+batch_size}: {e}")
            # Retry with smaller batch or skip
            continue
    
    return np.array(embeddings, dtype='float32')

# Generate all embeddings
print("\n" + "=" * 70)
print("GENERATING EMBEDDINGS")
print("=" * 70)

embeddings = generate_embeddings_batch(chunks, batch_size=100)

print(f"\n✓ Embeddings generated!")
print(f"  Shape: {embeddings.shape}")
print(f"  Size: {embeddings.nbytes / 1024 / 1024:.2f} MB")


GENERATING EMBEDDINGS
Generating embeddings in batches of 100...


Processing batches:   0%|          | 0/18 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 18/18 [00:38<00:00,  2.15s/it]


✓ Embeddings generated!
  Shape: (1747, 1536)
  Size: 10.24 MB


In [6]:
import pandas as pd
# Save embeddings
np.save('embeddings_openai.npy', embeddings)
print("✓ Saved embeddings to: embeddings_openai.npy")

# Save embedding metadata
embedding_metadata = {
    'model': 'text-embedding-3-small',
    'dimension': embeddings.shape[1],
    'total_embeddings': embeddings.shape[0],
    'date_created': pd.Timestamp.now().isoformat(),
}

with open('embedding_metadata.json', 'w') as f:
    json.dump(embedding_metadata, f, indent=2)

print("✓ Saved metadata to: embedding_metadata.json")

✓ Saved embeddings to: embeddings_openai.npy
✓ Saved metadata to: embedding_metadata.json


In [7]:
import faiss

print("=" * 70)
print("BUILDING FAISS INDEX")
print("=" * 70)

# Load embeddings if needed
embeddings = np.load('embeddings_openai.npy')

print(f"Embeddings shape: {embeddings.shape}")

# Normalize for cosine similarity
faiss.normalize_L2(embeddings)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine similarity)

# Add embeddings
index.add(embeddings)

print(f"\n✓ FAISS index created!")
print(f"  Total vectors: {index.ntotal}")
print(f"  Dimension: {dimension}")

# Save index
faiss.write_index(index, 'faiss_index.bin')
print("✓ Saved index to: faiss_index.bin")

BUILDING FAISS INDEX
Embeddings shape: (1747, 1536)

✓ FAISS index created!
  Total vectors: 1747
  Dimension: 1536
✓ Saved index to: faiss_index.bin


In [5]:
def retrieve_relevant_chunks(query, top_k=5):
    """
    Retrieve most relevant chunks for a query
    
    Args:
        query: User question
        top_k: Number of chunks to retrieve
    
    Returns:
        List of relevant chunks with scores
    """
    # Generate query embedding
    query_embedding = np.array([get_embedding(query)], dtype='float32')
    faiss.normalize_L2(query_embedding)
    
    # Search in FAISS
    scores, indices = index.search(query_embedding, top_k)
    
    # Format results
    results = []
    for idx, score in zip(indices[0], scores[0]):
        chunk = chunks[idx]
        results.append({
            'score': float(score),
            'content': chunk['content'],
            'metadata': chunk['metadata'],
            'chunk_id': chunk['chunk_id'],
            'tokens': chunk['tokens']
        })
    
    return results

# Test retrieval
print("\n" + "=" * 70)
print("TESTING RETRIEVAL")
print("=" * 70)

test_query = "Yüksek lisans programları neler?"
print(f"Query: {test_query}\n")

results = retrieve_relevant_chunks(test_query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"{i}. Score: {result['score']:.4f}")
    print(f"   Title: {result['metadata']['title']}")
    print(f"   URL: {result['metadata']['url']}")
    print(f"   Content: {result['content'][:200]}...")
    print()


TESTING RETRIEVAL
Query: Yüksek lisans programları neler?



NameError: name 'faiss' is not defined

In [29]:
def create_rag_prompt(query, retrieved_chunks, language='auto'):
    """
    Create prompt for OpenAI with retrieved context
    
    Args:
        query: User question
        retrieved_chunks: List of relevant chunks
        language: 'tr', 'en', or 'auto'
    
    Returns:
        Formatted prompt
    """
    # Detect language if auto
    if language == 'auto':
        # Simple detection: check for Turkish characters
        if any(char in query for char in 'ğüşıöçĞÜŞİÖÇ'):
            language = 'tr'
        else:
            language = 'en'
    
    # System prompt
    system_prompts = {
        'tr': """Sen İstanbul Sabahattin Zaim Üniversitesi için bir yardımcı asistansın. 
Öğrencilere ve ziyaretçilere üniversite hakkında bilgi veriyorsun.

Önemli kurallar:
1. Sadece verilen CONTEXT bilgilerini kullan
2. Bilmediğin bir şey sorulursa, bilmediğini söyle
3. Nazik ve profesyonel ol
4. Cevaplarında kaynak URL'lerini belirt
5. Türkçe karakter kullan (ı, ğ, ü, ş, ö, ç)""",
        
        'en': """You are an assistant for Istanbul Sabahattin Zaim University.
You help students and visitors with information about the university.

Important rules:
1. Only use information from the provided CONTEXT
2. If you don't know something, say you don't know
3. Be polite and professional
4. Include source URLs in your answers
5. Keep answers concise and clear
6. Detect the user’s question language.
7. Always translate the question into English internally for searching and retrieval.
8. Search Turkish data first.
9. If an answer is found → use it.
10. If no answer exists in Turkish, search the English data.
11. Regardless of the internal search language or source language, always answer in the user’s original language.
12. If the final answer was sourced from English data but the user asked in Turkish, translate it into natural Turkish before responding.
13. Never show the translation process to the user."""
    }
    
    # Build context from chunks
    context_parts = []
    for i, chunk in enumerate(retrieved_chunks, 1):
        context_parts.append(f"""
[KAYNAK {i}]
Başlık: {chunk['metadata']['title']}
URL: {chunk['metadata']['url']}
İçerik:
{chunk['content']}
""" if language == 'tr' else f"""
[SOURCE {i}]
Title: {chunk['metadata']['title']}
URL: {chunk['metadata']['url']}
Content:
{chunk['content']}
""")
    
    context = "\n---\n".join(context_parts)
    
    # User message
    user_message = f"""CONTEXT:
{context}

SORU: {query}

Lütfen yukarıdaki CONTEXT bilgilerini kullanarak soruyu yanıtla. Hangi kaynaktan aldığını belirt.""" if language == 'tr' else f"""CONTEXT:
{context}

QUESTION: {query}

Please answer the question using the CONTEXT above. Mention which source you used."""
    
    return system_prompts[language], user_message

# Test prompt creation
system, user = create_rag_prompt(test_query, results[:3], language='tr')

print("System Prompt:")
print(system)
print("\n" + "=" * 70)
print("\nUser Message:")
print(user[:500])
print("...")

System Prompt:
Sen İstanbul Sabahattin Zaim Üniversitesi için bir yardımcı asistansın. 
Öğrencilere ve ziyaretçilere üniversite hakkında bilgi veriyorsun.

Önemli kurallar:
1. Sadece verilen CONTEXT bilgilerini kullan
2. Bilmediğin bir şey sorulursa, bilmediğini söyle
3. Nazik ve profesyonel ol
4. Cevaplarında kaynak URL'lerini belirt
5. Türkçe karakter kullan (ı, ğ, ü, ş, ö, ç)


User Message:
CONTEXT:

[KAYNAK 1]
Başlık: Diploma, Sertifika ve Diğer İlgili Belgelerin Düzenlenmesine İlişkin Yönerge
URL: https://www.izu.edu.tr/izu-hakkinda/mevzuat/yonergeler/diploma-sertifika-ve-di%C4%9Fer-i-lgili-belgelerin-d%C3%BCzenlenmesine-i-li%C5%9Fkin-y%C3%B6nerge
İçerik:
Tanımlar 4- Bu Yönergede geçen; a) Dekan: Üniversitesinin Fakülte Dekanını, b) Hologram: Üniversitesine ait bilgileri taşıyan etiketi, c) Müdür: Üniversitesinin Yüksekokul Enstitü Müdürünü, ç) Rektör: Üniversitesi Rektörünü, d) 
...


In [10]:
def answer_question(query, top_k=5, language='auto', model='gpt-4o-mini'):
    """
    Complete RAG pipeline: Retrieve + Generate answer
    
    Args:
        query: User question
        top_k: Number of chunks to retrieve
        language: 'tr', 'en', or 'auto'
        model: OpenAI model to use
    
    Returns:
        dict with answer and sources
    """
    # Step 1: Retrieve relevant chunks
    print(f"🔍 Retrieving relevant information...")
    retrieved_chunks = retrieve_relevant_chunks(query, top_k)
    
    # Step 2: Create prompt
    system_prompt, user_message = create_rag_prompt(query, retrieved_chunks, language)
    
    # Step 3: Generate answer with OpenAI
    print(f"🤖 Generating answer with {model}...")
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        temperature=0.3,  # Lower = more factual
        max_tokens=1000,
    )
    
    answer = response.choices[0].message.content
    
    # Format output
    return {
        'query': query,
        'answer': answer,
        'sources': [
            {
                'title': chunk['metadata']['title'],
                'url': chunk['metadata']['url'],
                'relevance_score': chunk['score']
            }
            for chunk in retrieved_chunks
        ],
        'model': model,
        'total_tokens': response.usage.total_tokens
    }

# Test complete RAG
print("\n" + "=" * 70)
print("TESTING COMPLETE RAG PIPELINE")
print("=" * 70)

result = answer_question(
    "Yüksek lisans başvurusu nasıl yapılır?",
    top_k=3,
    language='tr'
)

print(f"\n📝 Soru: {result['query']}")
print(f"\n🤖 Cevap:\n{result['answer']}")
print(f"\n📚 Kaynaklar:")
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['title']}")
    print(f"     {source['url']}")
    print(f"     Skor: {source['relevance_score']:.4f}")
print(f"\n💰 Token kullanımı: {result['total_tokens']}")


TESTING COMPLETE RAG PIPELINE
🔍 Retrieving relevant information...
🤖 Generating answer with gpt-4o-mini...

📝 Soru: Yüksek lisans başvurusu nasıl yapılır?

🤖 Cevap:
Yüksek lisans başvurusu yapmak isteyen öğrencilerin en az dört yıllık lisans diplomasına sahip olmaları ve mevzuatta belirtilen diğer şartları sağlamaları gerekmektedir. Türkiye vatandaşı öğrencilerden dört yıllık lisans diplomasını yurt dışındaki yabancı üniversitelerden almış olanların diplomalarının denkliğinin Yükseköğretim Kurulu tarafından onaylanmış olması gerekmektedir. 

Lisansüstü programlara hakkını kazanan asıl ve yedek adayların listesi enstitü yönetim kurulu kararı ile kesinleşir ve sonuçlar enstitü müdürlüğü tarafından ilan edilir. Kayıtlar, akademik takvimde belirtilen günlerde yapılır. Kazanan adaylar, belirtilen süre içerisinde kesin kayıtlarını yaptırmalıdır. Süresi içinde kayıt yaptırmayan adaylar haklarını kaybeder ve bu adayların yerine başarılı bulunan yedek adaylar genel başarı değerlendirme sırası 

In [11]:
# Test questions
test_questions = [
    "Yüksek lisans programları neler?",
    "Üniversite ücretleri ne kadar?",
    "What are the tuition fees for international students?",
    "Burs imkanları var mı?",
    "Kampüste yurt var mı?",
]

print("=" * 70)
print("TESTING MULTIPLE QUESTIONS")
print("=" * 70)

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*70}")
    print(f"Test {i}/{len(test_questions)}")
    print(f"{'='*70}")
    
    result = answer_question(question, top_k=3)
    
    print(f"📝 {result['query']}")
    print(f"\n🤖 {result['answer']}")
    print(f"\n💰 Tokens: {result['total_tokens']}")
    print()

TESTING MULTIPLE QUESTIONS

Test 1/5
🔍 Retrieving relevant information...
🤖 Generating answer with gpt-4o-mini...
📝 Yüksek lisans programları neler?

🤖 Üzgünüm, ancak yukarıdaki CONTEXT bilgileri arasında İstanbul Sabahattin Zaim Üniversitesi'nin yüksek lisans programları hakkında spesifik bir bilgi bulunmamaktadır. Daha fazla bilgi almak için üniversitenin resmi web sitesini ziyaret etmenizi öneririm. 

Kaynak: [Graduate Education Institute Application](https://www.izu.edu.tr/en/academics/institute/graduate-education-institute/graduate)

💰 Tokens: 2021


Test 2/5
🔍 Retrieving relevant information...
🤖 Generating answer with gpt-4o-mini...
📝 Üniversite ücretleri ne kadar?

🤖 Üniversite ücretleri her yıl Mütevelli Heyeti tarafından maliyet artışları dikkate alınarak yeniden belirlenmektedir. İlan edilen ücretler yalnızca eğitim-öğretim masraflarını kapsamaktadır; konaklama, beslenme, ulaşım, kitap gibi diğer giderler bu ücretlerin dışında kalmaktadır. Kesin ücret bilgileri için üniversi

In [1]:
class RAGChatbot:
    """
    RAG Chatbot with conversation memory
    """
    def __init__(self, model='gpt-4o-mini'):
        self.model = model
        self.conversation_history = []
    
    def chat(self, query, top_k=5, language='auto'):
        """Chat with memory"""
        
        # Retrieve chunks
        retrieved_chunks = retrieve_relevant_chunks(query, top_k)
        
        # Create context
        system_prompt, _ = create_rag_prompt(query, retrieved_chunks, language)
        
        # Build messages with history
        messages = [{"role": "system", "content": system_prompt}]
        
        # Add conversation history (last 3 turns)
        messages.extend(self.conversation_history[-6:])
        
        # Add current query
        messages.append({"role": "user", "content": query})
        
        # Generate response
        response = openai.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.3,
            max_tokens=1000,
        )
        
        answer = response.choices[0].message.content
        
        # Update history
        self.conversation_history.append({"role": "user", "content": query})
        self.conversation_history.append({"role": "assistant", "content": answer})
        
        return {
            'answer': answer,
            'sources': retrieved_chunks,
            'tokens': response.usage.total_tokens
        }
    
    def reset(self):
        """Clear conversation history"""
        self.conversation_history = []

# Test chatbot with memory
chatbot = RAGChatbot()

print("=" * 70)
print("TESTING CHATBOT WITH MEMORY")
print("=" * 70)

# Conversation
questions = [
    "üniversitenin adı nedir?",
    "konumu nerde?",  # Follow-up question
    "nasıl gidebilirim?",  # Another follow-up
]

for q in questions:
    print(f"\n👤 Kullanıcı: {q}")
    result = chatbot.chat(q, top_k=3, language='tr')
    print(f"🤖 Asistan: {result['answer']}")
    print(f"💰 Tokens: {result['tokens']}")

TESTING CHATBOT WITH MEMORY

👤 Kullanıcı: üniversitenin adı nedir?


NameError: name 'retrieve_relevant_chunks' is not defined

In [13]:
# Save complete configuration
rag_config = {
    'embedding_model': 'text-embedding-3-small',
    'embedding_dimension': embeddings.shape[1],
    'llm_model': 'gpt-4o-mini',
    'total_chunks': len(chunks),
    'faiss_index_path': 'faiss_index.bin',
    'chunks_path': 'chunks.json',
    'embeddings_path': 'embeddings_openai.npy',
    'default_top_k': 5,
    'temperature': 0.3,
    'max_tokens': 1000,
}

with open('rag_config.json', 'w') as f:
    json.dump(rag_config, f, indent=2)

print("✓ Saved RAG configuration to: rag_config.json")

print("\n" + "=" * 70)
print("RAG SYSTEM COMPLETE! 🎉")
print("=" * 70)
print("\nFiles created:")
print("  1. embeddings_openai.npy - OpenAI embeddings")
print("  2. faiss_index.bin - Vector index")
print("  3. rag_config.json - System configuration")
print("\nYou can now:")
print("  - Use answer_question() for single queries")
print("  - Use RAGChatbot() for conversations")
print("  - Build a web interface (Streamlit/Gradio)")

✓ Saved RAG configuration to: rag_config.json

RAG SYSTEM COMPLETE! 🎉

Files created:
  1. embeddings_openai.npy - OpenAI embeddings
  2. faiss_index.bin - Vector index
  3. rag_config.json - System configuration

You can now:
  - Use answer_question() for single queries
  - Use RAGChatbot() for conversations
  - Build a web interface (Streamlit/Gradio)


In [16]:
RAGChatbot()

In [15]:
# Create a simple module for easy import
production_code = '''
import json
import numpy as np
import faiss
import openai
from dotenv import load_dotenv
import os

# Load config
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# Load resources
with open('chunks.json', 'r', encoding='utf-8') as f:
    chunks = json.load(f)

index = faiss.read_index('faiss_index.bin')

with open('rag_config.json', 'r') as f:
    config = json.load(f)

def answer_question(query, top_k=5):
    """Production RAG function"""
    # Your answer_question code here
    pass

# Ready to use!
'''

with open('rag_system.py', 'w') as f:
    f.write(production_code)

print("✓ Created rag_system.py for production use")

✓ Created rag_system.py for production use


In [30]:
result = answer_question("İstanbul Sabahattin Zaim Üniversitesi’nin yıllık lisans öğrenim ücreti ne kadar?", top_k=3)
    
print(f"📝 {result['query']}")
print(f"\n🤖 {result['answer']}")


🔍 Retrieving relevant information...
🤖 Generating answer with gpt-4o-mini...
📝 İstanbul Sabahattin Zaim Üniversitesi’nin yıllık lisans öğrenim ücreti ne kadar?

🤖 Üzgünüm, İstanbul Sabahattin Zaim Üniversitesi'nin yıllık lisans öğrenim ücreti hakkında elimde bilgi bulunmamaktadır. Ancak lisansüstü eğitim ücretleri ve burslar hakkında detaylı bilgiye [bu linkten](https://www.izu.edu.tr/akademik/enstitu/lisansustu-egitim-enstitusu/ucretler-burs) ulaşabilirsiniz. Başka bir konuda yardımcı olmamı ister misiniz?
